# Run Evaluations on our RAG chatbot! 

<div align="center">
    <p style="text-align:left">
        <img alt="phoenix logo" src="https://repository-images.githubusercontent.com/564072810/f3666cdf-cb3e-4056-8a25-27cb3e6b5848" width="800"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</div>

## Let's get started! 

In [1]:
%pip install -qqq "arize-phoenix==11.21.0" "openai>=1" nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from getpass import getpass
import phoenix as px
import nest_asyncio

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

nest_asyncio.apply()

<img alt="Document Retrieval Evaluation Image" src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/phoenix-docs-images/documentRelevanceDiagram.png" width="1000"/>

In [15]:
from phoenix.session.evaluation import get_retrieved_documents
retrieved_documents_df = get_retrieved_documents(px.Client(), project_name="our-rag-project", timeout=None)
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
5fe3e8fd6f82bfe9 0                  6dc8afa6bcdc69fa2e3efa818fd69bf5   
                 1                  6dc8afa6bcdc69fa2e3efa818fd69bf5   
                 2                  6dc8afa6bcdc69fa2e3efa818fd69bf5   
                 3                  6dc8afa6bcdc69fa2e3efa818fd69bf5   
                 4                  6dc8afa6bcdc69fa2e3efa818fd69bf5   
a001cfe3483647d5 0                  05b736da6a425648d3d31dd825698bec   
                 1                  05b736da6a425648d3d31dd825698bec   
                 2                  05b736da6a425648d3d31dd825698bec   
                 3                  05b736da6a425648d3d31dd825698bec   
                 4                  05b736da6a425648d3d31dd825698bec   
c66e574ea9fe69ba 0                  76935651a31f75845ae1cc058495bc0b   
                 1                  76935651a31f75845ae1cc058495bc0b   
                 2                  76935651a31f75845ae1cc058495bc0b   
                 3                  76935651a31f75845ae1cc058495bc0b   
                 4                  76935651a31f75845ae1cc058495bc0b   

                                            input  \
context.span_id  document_position                  
5fe3e8fd6f82bfe9 0                    course name   
                 1                    course name   
                 2                    course name   
                 3                    course name   
                 4                    course name   
a001cfe3483647d5 0                    course name   
                 1                    course name   
                 2                    course name   
                 3                    course name   
                 4                    course name   
c66e574ea9fe69ba 0                  course topics   
                 1                  course topics   
                 2                  course topics   
                 3                  course topics   
                 4                  course topics   

                                                                            reference  
context.span_id  document_position                                                     
5fe3e8fd6f82bfe9 0                  The purpose of this document is to capture fre...  
                 1                  Yes, we will keep all the materials after the ...  
                 2                  GitHub - DataTalksClub data-engineering-zoomca...  
                 3                  You can start by installing and setting up all...  
                 4                  Yes, even if you don't register, you're still ...  
a001cfe3483647d5 0                  The purpose of this document is to capture fre...  
                 1                  Yes, we will keep all the materials after the ...  
                 2                  GitHub - DataTalksClub data-engineering-zoomca...  
                 3                  You can start by installing and setting up all...  
                 4                  Yes, even if you don't register, you're still ...  
c66e574ea9fe69ba 0                  The purpose of this document is to capture fre...  
                 1                  Yes, we will keep all the materials after the ...  
                 2                  GitHub - DataTalksClub data-engineering-zoomca...  
                 3                  You can start by installing and setting up all...  
                 4                  Yes, even if you don't register, you're still ...

In [16]:
from phoenix.session.evaluation import get_qa_with_reference

queries_df = get_qa_with_reference(px.Client(), project_name="our-rag-project", timeout=None)
queries_df

,input,output,reference
context.span_id,,,
ce2272c6d7fed335,What is the name of this course?,The name of this course is **Data Engineering ...,The purpose of this document is to capture fre...
a5b6a984a3db453f,What is the name of this course?,"The name of the course is ""Data Engineering Zo...",The purpose of this document is to capture fre...
e78d158a6aa21457,What topics will be covered?,It appears that the specific topics covered in...,The purpose of this document is to capture fre...


In [11]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)

eval_model = OpenAIModel(model="gpt-4")
relevance_evaluator = RelevanceEvaluator(eval_model)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_evaluator = QAEvaluator(eval_model)

In [17]:
retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]
retrieved_documents_relevance_df

run_evals |          | 0/15 (0.0%) | ⏳ 00:00<? | ?it/s

label  score  \
context.span_id  document_position                     
5fe3e8fd6f82bfe9 0                  unrelated      0   
                 1                  unrelated      0   
                 2                  unrelated      0   
                 3                  unrelated      0   
                 4                  unrelated      0   
a001cfe3483647d5 0                  unrelated      0   
                 1                  unrelated      0   
                 2                  unrelated      0   
                 3                  unrelated      0   
                 4                  unrelated      0   
c66e574ea9fe69ba 0                  unrelated      0   
                 1                  unrelated      0   
                 2                  unrelated      0   
                 3                  unrelated      0   
                 4                  unrelated      0   

                                                                          explanation  
context.span_id  document_position                                                     
5fe3e8fd6f82bfe9 0                  The question is asking for the name of the cou...  
                 1                  The question is asking for the name of the cou...  
                 2                  The reference text mentions a GitHub repositor...  
                 3                  The question is asking for the name of a cours...  
                 4                  The question is asking for the name of a cours...  
a001cfe3483647d5 0                  The question asks for the 'course name'. Howev...  
                 1                  The question is asking for the name of the cou...  
                 2                  The reference text mentions a GitHub repositor...  
                 3                  The question is asking for the name of a cours...  
                 4                  The question is asking for the name of a cours...  
c66e574ea9fe69ba 0                  The question is asking for the topics of a cou...  
                 1                  The question is asking for the topics of the c...  
                 2                  The reference text is a GitHub link to a data ...  
                 3                  The question is asking for information about '...  
                 4                  The question is asking for information about t...

In [18]:
hallucination_eval_df, qa_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_evaluator],
    provide_explanation=True,
    concurrency=20,
)
hallucination_eval_df

run_evals |          | 0/6 (0.0%) | ⏳ 00:00<? | ?it/s

,label,score,explanation
context.span_id,,,
ce2272c6d7fed335,factual,0,The query asks for the name of the course. The...
a5b6a984a3db453f,hallucinated,1,The answer states that the name of the course ...
e78d158a6aa21457,hallucinated,1,The answer is hallucinated because it provides...


In [19]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_eval_df),
    DocumentEvaluations(
        eval_name="Retrieval Relevance", dataframe=retrieved_documents_relevance_df
    ),
)